In [39]:
import pandas as pd

In [40]:
import requests

def descargar_archivo(url: str, anio: str):
    response = requests.get(url)
    nombre_archivo = './datos/descargados/calidad-aire-diario-' + anio + '.csv'
    if response.status_code == 200:
        with open(nombre_archivo, "wb") as f:
            f.write(response.content)
        print(f"Archivo descargado correctamente como '{nombre_archivo}'")
    else:
        print(f"Error al descargar el archivo '{nombre_archivo}'. Código de estado: {response.status_code}")

In [41]:
df_estaciones = pd.read_csv('./datos/estaciones-madrid.csv', sep=';')
df_estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PUNTO_MUESTREO  53 non-null     int64  
 1   LONGITUD        53 non-null     float64
 2   LATITUD         53 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.4 KB


In [42]:
def procesar_archivo(anio: str):

    df = pd.read_csv('./datos/descargados/calidad-aire-diario-' + anio + '.csv', sep=';')

    df = df[df['MAGNITUD'] == 8]

    dias_cols = [col for col in df.columns if col.startswith('D') and len(col) == 3]
    df_long = df.melt(
        id_vars=['PUNTO_MUESTREO', 'ANO', 'MES', 'MAGNITUD'],
        value_vars=dias_cols,
        var_name='DIA',
        value_name='VALOR'
    )

    df_long['DIA_NUM'] = df_long['DIA'].str.extract('D(\d+)').astype(int)
    df_long['FECHA'] = pd.to_datetime(
        df_long['ANO'].astype(str) + '-' + 
        df_long['MES'].astype(str).str.zfill(2) + '-' + 
        df_long['DIA_NUM'].astype(str).str.zfill(2),
        errors='coerce'
    )

    df_long = df_long.dropna(subset=['FECHA'])
    df_long = df_long[pd.to_numeric(df_long['VALOR'], errors='coerce').notnull()]
    df_long['VALOR'] = df_long['VALOR'].astype(float)
    df_long = df_long[df_long['VALOR'] >= 0]

    df_media_mensual = (
        df_long
        .groupby(['ANO', 'MES', 'PUNTO_MUESTREO'])['VALOR']
        .mean()
        .reset_index()
    )

    df_media_mensual['FECHA'] = df_media_mensual['ANO'].astype(str) + '-' + df_media_mensual['MES'].astype(str).str.zfill(2)

    df_media_mensual['PUNTO_MUESTREO'] = df_media_mensual['PUNTO_MUESTREO'].astype(str).str[:8]
    df_media_mensual['PUNTO_MUESTREO'] = df_media_mensual['PUNTO_MUESTREO'].astype(int)

    df_media_mensual = pd.merge(df_media_mensual, df_estaciones[['PUNTO_MUESTREO', 'LONGITUD', 'LATITUD']], on='PUNTO_MUESTREO', how='left')

    df_final = df_media_mensual[['FECHA', 'VALOR', 'PUNTO_MUESTREO', 'LONGITUD', 'LATITUD']]
    df_final = df_final.sort_values(by='FECHA')

    nombre_archivo = './datos/procesados/calidad-aire-diario-' + anio + '.csv'

    df_final.to_csv(nombre_archivo, index=False)

    print(f"Archivo procesado correctamente como '{nombre_archivo}'")

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_7383/3493282381.py:15: SyntaxWarning: invalid escape sequence '\d'
  df_long['DIA_NUM'] = df_long['DIA'].str.extract('D(\d+)').astype(int)


In [43]:
def procesar_anio(url: str, anio: str):
    #descargar_archivo(url, anio)
    procesar_archivo(anio)
    #print(f"Año procesado correctamente '{anio}'")

In [44]:
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306604-calidad-aire-diario.csv', '2001')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306602-calidad-aire-diario.csv', '2002')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306600-calidad-aire-diario.csv', '2003')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306598-calidad-aire-diario.csv', '2004')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306596-calidad-aire-diario.csv', '2005')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306594-calidad-aire-diario.csv', '2006')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306592-calidad-aire-diario.csv', '2007')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306590-calidad-aire-diario.csv', '2008')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306588-calidad-aire-diario.csv', '2009')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306586-calidad-aire-diario.csv', '2010')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306584-calidad-aire-diario.csv', '2011')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306582-calidad-aire-diario.csv', '2012')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306580-calidad-aire-diario.csv', '2013')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306578-calidad-aire-diario.csv', '2014')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306576-calidad-aire-diario.csv', '2015')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306574-calidad-aire-diario.csv', '2016')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-7775098-calidad-aire-diario.csv', '2017')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-7775096-calidad-aire-diario.csv', '2018')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306606-calidad-aire-diario.csv', '2019')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306609-calidad-aire-diario.csv', '2020')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306612-calidad-aire-diario.csv', '2021')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306615-calidad-aire-diario.csv', '2022')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306618-calidad-aire-diario.csv', '2023')
procesar_anio('https://datos.madrid.es/egob/catalogo/201410-10306621-calidad-aire-diario.csv', '2024')

Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2001.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2002.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2003.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2004.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2005.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2006.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2007.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2008.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2009.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2010.csv'
Archivo procesado correctamente como './datos/procesados/calidad-aire-diario-2011.csv'
Archivo procesado correctamente como './dat

In [45]:
import glob
import os

carpeta = './datos/procesados/'
csv_files = sorted(glob.glob(os.path.join(carpeta, '*.csv')))

dfs = [pd.read_csv(f) for f in csv_files]
df_concat = pd.concat(dfs, ignore_index=True)

nombre_csv = './datos/mapa-evolucion-contaminacion-aire-madrid-2024.csv'

df_concat.to_csv(nombre_csv, index=False, encoding='utf-8')

print(f"Archivo procesado correctamente como '{nombre_csv}'")

json_str = df_concat.to_json(orient='records', force_ascii=False)

nombre_js = 'mapa-evolucion-contaminacion-aire-madrid-2024.js'

with open('mapa-evolucion-contaminacion-aire-madrid-2024.js', 'w', encoding='utf-8') as f:
    f.write(f'const datos = {json_str};\n')

print(f"Archivo procesado correctamente como '{nombre_js}'")

Archivo procesado correctamente como './datos/mapa-evolucion-contaminacion-aire-madrid-2024.csv'
Archivo procesado correctamente como 'mapa-evolucion-contaminacion-aire-madrid-2024.js'


In [46]:
df_concat.describe()

,VALOR,PUNTO_MUESTREO,LONGITUD,LATITUD
count,5600.000000,5.600000e+03,5600.000000,5600.000000
mean,38.690833,2.807903e+07,-3.680008,40.432378
std,16.312328,1.746497e+01,0.047630,0.040139
min,0.600000,2.807900e+07,-3.774444,40.346944
25%,26.090438,2.807902e+07,-3.711389,40.408056
50%,37.207143,2.807904e+07,-3.688333,40.424167
75%,49.573733,2.807905e+07,-3.651389,40.465000
max,109.451613,2.807906e+07,-3.575000,40.518056


In [51]:
import plotly.express as px

fig = px.scatter(
    df_concat,
    x='FECHA',
    y='VALOR',
    color='VALOR',
    title='Evolución de valores por estación a lo largo del tiempo',
    labels={'FECHA': 'Fecha', 'VALOR': 'Valor', 'PUNTO_MUESTREO': 'Estación'},
    hover_data=['PUNTO_MUESTREO', 'VALOR']
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()